In [1]:
import sys
from pathlib import Path

In [2]:
module_path = Path.cwd().parents[2]
if module_path not in sys.path:
    sys.path.append(str(module_path))

In [3]:
module_path

PosixPath('/home/marek/Projects/Python/evaluating-student-writing')

In [32]:
import spacy
from spacy.scorer import Scorer
from spacy.tokens import Doc, DocBin
from spacy.training import Example

In [4]:
from src.loader import TextLoader
from src.model import DatasetType

In [6]:
loader = TextLoader(dataset_type=DatasetType.V1_WITH_PREDICTIONSTRING)
nlp = spacy.load("models/spacy_resume/model-best/")

In [7]:
text = loader.load_random_text(purify_discourses=True, purify_text=True)
text.id

'FFC43F453EF6'

In [11]:
print(text)

Senador first I want to say that I dissagree with the idea of the Electoral college becouse the people needs vote for the president that they want that's democrastic. One of the reasons is that the Electoral College is unfair outdated and irrational. The Electoral College is widely regarded as an anachorsims not democratic method of selecting a president that ought to oberruled by declaring the candidate who recieve the most popular votes the winners. Its entirely possible that the winner of the electoral vote will not win the nation popular vote. There's a few reasons for retainig the Electoral College despite its lack of democratic pedigree the first one is that a dispute over the outcome of an Electoral College vote is possible but it's possible but it's likely that a disputate over the popular vote. Other is that the Electoral College requires a presidential candidate to have trans regional appeal. The Electoral College avoids the problem of elections in wich no candidate recieves 

In [12]:
for disc in text.discourses:
    print(disc)

--- 1616005177466 (33 -> 83 | 7 -> 15) - Position ---
I dissagree with the idea of the Electoral college
-----------------------------------------------------
--- 1616005188474 (92 -> 166 | 17 -> 28) - Claim ---
the people needs vote for the president that they want that's democrastic.
----------------------------------------------------
--- 1616005211426 (194 -> 225 | 35 -> 39) - Claim ---
the Electoral College is unfair
-----------------------------------------------------
--- 1616005219233 (226 -> 234 | 40 -> 40) - Claim ---
outdated
-----------------------------------------------------
--- 1616005225721 (239 -> 250 | 42 -> 42) - Claim ---
irrational.
-----------------------------------------------------
--- 1616005521600 (251 -> 553 | 43 -> 91) - Evidence ---
The Electoral College is widely regarded as an anachorsims not democratic method of selecting a president that ought to oberruled by declaring the candidate who recieve the most popular votes the winners. Its entirely possible

In [9]:
doc = nlp(text.text)

In [10]:
for token in doc.ents:
    print(token.text, token.label_, token.start_char, token.end_char)

college DE 76 83
democrastic DE 154 165
One DS 167 170
unfair DE 219 225
outdated DS 226 234
irrational DS 239 249
The DS 251 254
vote DE 548 552
There DS 554 559
's DS 559 561
appeal DE 908 914
The DS 916 919
cast DE 1024 1028
For DS 1030 1033
want DE 1165 1169


In [13]:
word_idx = []
for disc in text.discourses:
    word_idx.extend((disc.predictionstring[0], disc.predictionstring[-1]))

ents = []

DS_token = "B-DS"
DE_token = "B-DE"
use_DS = True
for ind, word in enumerate(text.words):
    if use_DS:
        curr_token = DS_token
    else:
        curr_token = DE_token

    if ind in word_idx:
        ents.append(curr_token)
        use_DS = not use_DS
    else:
        ents.append("O")


manual_doc = Doc(nlp.vocab, text.words, ents=ents)


In [14]:
spacy.displacy.render(manual_doc, style="ent", jupyter=True)


In [15]:
spacy.displacy.render(doc, style="ent", jupyter=True)

In [34]:
scorer = Scorer()

doc_bin = DocBin().from_disk("data/NER_test.spacy")
examples = []
for doc in doc_bin.get_docs(nlp.vocab):
    examples.append(Example.from_dict(doc, {"entities": doc.ents}))

scorer.score(examples)


{'token_acc': 1.0,
 'token_p': 1.0,
 'token_r': 1.0,
 'token_f': 1.0,
 'sents_p': None,
 'sents_r': None,
 'sents_f': None,
 'tag_acc': None,
 'pos_acc': None,
 'morph_acc': None,
 'morph_micro_p': None,
 'morph_micro_r': None,
 'morph_micro_f': None,
 'morph_per_feat': None,
 'dep_uas': None,
 'dep_las': None,
 'dep_las_per_type': None,
 'ents_p': 1.0,
 'ents_r': 1.0,
 'ents_f': 1.0,
 'ents_per_type': {'DS': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'DE': {'p': 1.0, 'r': 1.0, 'f': 1.0}},
 'cats_score': 0.0,
 'cats_score_desc': 'macro F',
 'cats_micro_p': 0.0,
 'cats_micro_r': 0.0,
 'cats_micro_f': 0.0,
 'cats_macro_p': 0.0,
 'cats_macro_r': 0.0,
 'cats_macro_f': 0.0,
 'cats_macro_auc': 0.0,
 'cats_f_per_type': {},
 'cats_auc_per_type': {}}